In [56]:
#와인 가격 계산
#numpredict.py
from random import random,randint
import math

def wineprice(rating,age):
  '''
  rating  등급
  age 나이
  '''
  peak_age=rating-50
  
  # 등급에 따라 가격 결정
  price=rating/2
  if age>peak_age:
    # 최적년도가 지나면 가격이 5년단위로 저하
    price=price*(5-(age-peak_age))
  else:
    #최적년도에 근접하면 원래의 5배의 가격
    price=price*(5*((age+1)/peak_age))
  if price<0: price=0
  return price


def wineset1():
  rows=[]
  for i in range(300):
    # 무작위로 나이와 등급을 생성
    rating=random()*50+50
    age=random()*50

    # 가격
    price=wineprice(rating,age)
    
    # 노이즈를 부가 한다.
    price*=(random()*0.4+0.8)

    # 데이터 추가
    rows.append({'input':(rating,age),
                 'result':price})
  return rows




In [57]:
wineprice(95.0,3.0)

wineprice(95.0,8.0)

wineprice(99.0,1.0)

data = wineset1()

data

[{'input': (62.8510697930621, 7.6822235920128765),
  'result': 94.10443685347214},
 {'input': (73.94816587505609, 46.45851791190238), 'result': 0.0},
 {'input': (86.90118364235832, 14.182142510739482),
  'result': 71.81995202294009},
 {'input': (91.27179649574259, 13.351718958806208),
  'result': 68.15735533479277},
 {'input': (56.36725492037915, 49.02949861770619), 'result': 0.0},
 {'input': (78.45470810039333, 21.848123647642904),
  'result': 131.37400898306043},
 {'input': (91.9874471367654, 7.364654249002656),
  'result': 39.389721064838476},
 {'input': (80.44882411941728, 6.059771627748689),
  'result': 55.76578755478634},
 {'input': (51.12007747566039, 43.03145718041767), 'result': 0.0},
 {'input': (68.54003186064023, 18.264760715683675),
  'result': 160.84900789662132},
 {'input': (54.27486549786272, 40.12705141843091), 'result': 0.0},
 {'input': (88.14645565765065, 7.202241242450919), 'result': 52.6468581486772},
 {'input': (62.245795350674555, 10.451392466688686),
  'result': 

In [59]:
def euclidean(v1,v2):
  d=0.0
  for i in range(len(v1)):
    d+=(v1[i]-v2[i])**2
  return math.sqrt(d)

#euclidean(data[0]['input'],data[1]['input'])
#euclidean(data[3]['input'],data[1]['input'])

def getdistances(data,vec1):
  '''
  거리 리스트를 얻는다.
  '''
  distancelist=[]
  for i in range(len(data)):
    vec2=data[i]['input']
    distancelist.append((euclidean(vec1,vec2),i))
  distancelist.sort()
  return distancelist

def knnestimate(data,vec1,k=5):
  # Get sorted distances
  dlist=getdistances(data,vec1)
  avg=0.0
  
  # 상위 k개의 결과 평균을 얻음.
  for i in range(k):
    idx=dlist[i][1]
    avg+=data[idx]['result']
  avg=avg/k
  return avg


* k 개수 데이터가 많으면 많은 만큼커야 좀 더 신뢰할 수 있지 않나

In [60]:
print(knnestimate( data , (95.0 , 3.0 ) ))
print(knnestimate( data , (99.0 , 3.0 ) ))

print(knnestimate( data , (99.0 , 5.0 ) ))

print(wineprice( 99.0 , 5.0 ))

print(knnestimate( data,(99.0 , 5.0 ),k=1))


24.164457244509308
20.695400064411995
36.27500655231254
30.306122448979593
24.73141908194826


In [61]:
def inverseweight(dist,num=1.0,const=0.1):
  return num/(dist+const)

def subtractweight(dist,const=1.0):
  if dist>const: 
    return 0
  else: 
    return const-dist

def gaussian(dist,sigma=5.0):
  return math.e**(-dist**2/(2*sigma**2))


In [64]:
subtractweight(0.1)
inverseweight(0.1)
gaussian(0.1)

0.9998000199986667

In [69]:
def weightedknn(data,vec1,k=5,weightf=gaussian):
  '''
  가중치 적용 knn
  '''
  dlist=getdistances(data,vec1)
  avg=0.0
  totalweight=0.0
  
  # Get weighted average
  for i in range(k):
    dist=dlist[i][0]
    idx=dlist[i][1]
    weight=weightf(dist)
    avg+=weight*data[idx]['result']
    totalweight+=weight
  if totalweight==0: return 0
  avg=avg/totalweight
  return avg


In [70]:
def dividedata(data,test=0.05):
  '''
  테스트 셋 나누기
  '''    
  trainset=[]
  testset=[]
  for row in data:
    if random()<test:
      testset.append(row)
    else:
      trainset.append(row)
  return trainset,testset

def testalgorithm(algf,trainset,testset):
  error=0.0
  for row in testset:
    guess=algf(trainset,row['input'])
    error+=(row['result']-guess)**2
    #print row['result'],guess
  #print error/len(testset)
  return error/len(testset)

def crossvalidate(algf,data,trials=100,test=0.1):
  error=0.0
  for i in range(trials):
    trainset,testset=dividedata(data,test)
    error+=testalgorithm(algf,trainset,testset)
  return error/trials
